### <center>Zadanie 4</center>
#### Grupowanie na zbiorze dotyczącym marskości wątroby

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from gower import gower_matrix
from kmodes.kprototypes import KPrototypes
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import mutual_info_score, normalized_mutual_info_score
from sklearn.metrics import rand_score
from sklearn.metrics import silhouette_score
from sklearn.metrics.cluster import contingency_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

### Wczytanie danych

In [ ]:
data = pd.read_csv('cirrhosis.csv')

status = data.Status
status_encoder = LabelEncoder()
status_encoder.fit(status)
status = status_encoder.transform(status)

data.drop(columns=['ID', 'Status'], inplace=True)
data['Stage'] = data['Stage'].astype('category')

data.info()

In [ ]:
data.head()

### Wizualizacja danych

<ol>
<li>wykresy kołowe</li>
<li>histogramy</li>
<li>macierz korelacji</li>
</ol>

### Przygotowanie danych

<ol>
<li>konwersja object do category</li>
<li>uzupełnienie danych</li>
<li>standard scaler na kolumnach float, int</li>
</ol>

#### Wykresy kołowe przedstawiają podział pacjentów według:
<ol>
<li>przyjętego lekarstwa</li>
<li>płci</li>
<li>obecności wodobrzusza</li>
<li>obecności hepatomegalii</li>
<li>obecności pajączków naczyniowych</li>
<li>obecność obrzęku w połączeniu z leczeniem diuretykami</li>
</ol>

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(12, 16))
plt.suptitle('Podział pacjentów według kategorii', fontsize=20)

colors = plt.cm.Set3.colors
titles = ['Przyjmowanie penicylaminy', 'Płeć', 'Wodobrzusze', 'Hepatomegalia', 'Pajączki naczyniowe', 'Obrzęk z leczeniem diuretykami', 'Stopień marskości wątroby']
object_columns = data.select_dtypes(include=['object']).columns.values
object_columns = np.append(object_columns, 'Stage')

default_mapping = {'Y': 'Tak', 'N': 'Nie', 'nan': 'Nie podano'}

for i, column in enumerate(object_columns):
    x, y = divmod(i, 2)
    el = data.groupby(column, dropna=False, as_index=True, observed=True).size()
    explode = [0.05 for _ in range(len(el))]

    if column == 'Sex':
        label_mapping = {'F': 'Kobieta', 'M': 'Mężczyzna'}
        labels = [label_mapping.get(str(label), str(label)) for label in el.index]

    elif column == 'Edema':
        label_mapping = {
            'N': 'Brak',
            'S': 'Reaguje na leczenie',
            'Y': 'Nie reaguje na leczenie',
            'nan': 'Nie podano'
        }
        labels = [
            label_mapping.get(str(label), 'Nie podano') if pd.isna(label)
            else label_mapping.get(str(label), str(label))
            for label in el.index
        ]

    else:
        labels = [
            default_mapping.get(str(label), 'Nie podano') if pd.isna(label)
            else default_mapping.get(str(label), str(label))
            for label in el.index
        ]

    ax[x, y].pie(
        x=el,
        explode=explode,
        labels=labels,
        colors=colors,
        autopct='%1.1f%%',
        textprops={'fontsize': 12, 'fontweight': 'bold'},
        radius=1,
        startangle=180,
        labeldistance=1.1,
        wedgeprops={'edgecolor': 'white', 'linewidth': 2},
        normalize=True,
    )
    ax[x, y].set_title(titles[i], fontsize=15, fontweight='bold')

ax[3, 1].axis('off')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### Ułamek wartości NaN w poszczególnych kolumnach

In [ ]:
round(data.isna().sum()/len(data), 2)

In [ ]:
columns_to_drop = [column for column in data.columns if (data[column].isna().sum() / len(data)) * 100 > 30]
data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
round(data.isna().sum()/len(data), 2)

#### Konwersja object na category + Uzupełnienie danych kategorycznych

Uzupełnienie wartości następuje losowo zachowując procenty udziału tych kategorii w liczebności bez NaN.

In [ ]:
encoders = [LabelEncoder() for _ in range(len(object_columns))]

for i, column in enumerate(object_columns):
    counts = data[column].value_counts()
    missing_count = data[column].isna().sum()
    counts *= (missing_count / counts.sum())
    counts = np.ceil(counts).astype(int)
    series = pd.Series(np.repeat(counts.index, counts))[:missing_count]
    series = series.sample(frac=1, random_state=42).reset_index(drop=True)
    data.loc[data[column].isna(), column] = series.values
    data[column] = encoders[i].fit_transform(data[column])
    data[column] = data[column].astype('category')

data.info()

#### Histogramy przedstawiają rozkład danych w kolumnach:

<ol>
<li>N_Days -> liczby dni między rejestracją, a wcześniejszym zgonem, przeszczepem lub końcem analizy badania</li>
<li>Age -> wiek pacjenta w dniach</li>
<li>Bilirubin -> stężenie bilirubiny (mg/dl)</li>
<li>Choresterol -> stężenie choresterolu (mg/dl)</li>
<li>Albumin -> albumina (mg/dl)</li>
<li>Copper -> ilość miedzi w moczu (µg/dzień)</li>
<li>Alk_Phos -> fosfataza alkaliczna (U/litr)</li>
<li>SGOT -> aminotransferaza asparaginianowa (U/ml)</li>
<li>Tryglicerides -> liczba triglicerydów</li>
<li>Platelets -> liczba płytek krwi na ml/1000</li>
<li>Prothrombin -> czas protrombinowy -> czas krzepnięcia krwi</li>
</ol>

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(16, 12))
plt.suptitle('Rozkład wartości w kolumnach z zmiennymi liczbowymi', fontsize=20)

numeric_columns = data.select_dtypes(include='number').columns

unit_mapping = {
    'N_Days': 'Dni',
    'Age': 'Dni',
    'Bilirubin': 'mg/dl',
    'Cholesterol': 'mg/dl',
    'Albumin': 'mg/dl',
    'Copper': 'µg/dzień',
    'Alk_Phos': 'U/litr',
    'SGOT': 'U/ml',
    'Tryglicerides': 'mg/dl',
    'Platelets': 'tys./ml',
    'Prothrombin': 'sekundy'
}

for i, column in enumerate(numeric_columns):
    x, y = divmod(i, 3)
    ax[x, y].hist(
        data[column],
        bins=20,
        color='#4C72B0',
        edgecolor='black',
        alpha=0.85
    )

    ax[x, y].set_title(column, fontsize=14, fontweight='bold')
    ax[x, y].set_xlabel('Wartość', fontsize=11)
    ax[x, y].set_ylabel('Liczba obserwacji', fontsize=11)
    ax[x, y].grid(True, linestyle='--', alpha=0.6)

    jednostka = unit_mapping.get(column)
    if jednostka:
        ax[x, y].legend([f'Jednostka: {jednostka}'], loc='upper right', fontsize=10)

for i in range(len(numeric_columns), 12):
    x, y = divmod(i, 3)
    ax[x, y].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### Uzupełnienie danych numerycznych

Wykorzystanie mediany dla małych braków i interpolacji w reszcie przypadków.
Zastosowanie StandardScalera dla zminiejszenia odległości między punktami w zbiorze (ułatwi to robotę dla kMeans).

In [ ]:
number_columns = data.select_dtypes(exclude='category').columns

for i, column in enumerate(number_columns):
    data[column] = data[column].fillna(data[column].median())

standardizer = StandardScaler()
after_standard = standardizer.fit_transform(data[number_columns])
data.drop(columns=number_columns, inplace=True)
data = pd.concat([data, pd.DataFrame(after_standard, columns=number_columns)], axis=1)

data.info()

In [ ]:
category_columns = data.select_dtypes(include='category').columns

In [ ]:
categorical_indices = [data.columns.get_loc(col) for col in category_columns]

In [ ]:
cat_features_array = np.zeros(data.shape[1], dtype=bool) 
cat_features_array[categorical_indices] = True

#### Macierz korelacji Pearsona

In [ ]:
plt.figure(figsize=(12, 10))
correlation_mat = data.corr()
plt.imshow(
    X=correlation_mat,
    cmap='viridis',
    vmax=1,
    vmin=-1,
)
plt.colorbar(label='Wartość korelacji')
plt.xticks(ticks=np.arange(len(correlation_mat.columns)), labels=correlation_mat.columns, rotation=45)
plt.yticks(ticks=np.arange(len(correlation_mat.columns)), labels=correlation_mat.columns)
plt.title('Macierz korelacji dla zbioru marskości wątroby', fontsize=14, pad=20)

for i in range(len(correlation_mat.columns)):
    for j in range(len(correlation_mat.columns)):
        text = f'{correlation_mat.iloc[i, j]:.2f}'
        if correlation_mat.iloc[i, j] >= 0.2:
            plt.text(j, i, text, ha='center', va='center', color='white', fontsize=12)

plt.grid(False)

### <center>Analiza skupień przy wykorzystaniu algorytmu KPrototypes, AgglomerativeClustering</center>

<b>KPrototypes</b> - jest to metoda łączaca działanie KModes i KMeans w zależności od typu danych. Dla cech numerycznych wykorzystuje KMeans z dedykowaną metryką odległości (miara czebyszewa, euklidesowa, manhattan). Dystans dla zmiennych kategorycznych jest obliczany jak w metodzie KModes.

<b>AgglomerativeClustering</b> - metoda hierarchiczna, która działa od dołu do góry. Na początku każdy punkt jest traktowany jako osobna grupa i potem na podstawie wybranego linkage klastry są łączone w większe.
Możliwe wartości linkage:
- Ward (połączenie Ward'a) -> metoda łącząca skupienia na zasadzie minimalizacji sumy kwadratów odległości pomiędzy członkami oraz środkiem klastra.
- Maximum/Complete linkage (kompletne łączenie klastrów) -> minimalizowanie maksymalnej odległości pomiędzy parami obserwacji klastrów.
- Average linkage (średnie połączenie klastrów) -> minimalizowanie średniej odległości pomiędzy parami obserwacji klastrów.
- Single linkage (pojedyncze łączenie klastrów) -> minimalizowanie minimalnej odległości między pomiędzy parami obserwacji klastrów.

Poniżej znajduje się metodologia testowania obu metod:
<ol>
<li>Metoda łokcia</li>
<li>Dendrogram</li>
<li>Kryteria wewnętrzne</li>
<ul>
<li>Kryterium współczynnika wariancji Calińskiego-Harabasza (VCR)</li>
<li>Współczynnik zarysu (ang. Silhouette coefficient)</li>
</ul>
<li>Kryteria zewnętrzne</li>
<ul>
<li>Macierz kontyngencji</li>
<li>Indeks Randa</li>
<li>Współczynnik wzajemnej informacji (ang. Mutual Information)</li>
</ul>
</ol>

#### Metryka Gowera (Gower Distance)

Do obliczenia odległości pomiędzy obserwacjami w zbiorze danych mieszanych (zawierających zarówno zmienne numeryczne, jak i kategoryczne) wykorzystano metrykę Gowera. Jest to miara podobieństwa zaprojektowana specjalnie do takich sytuacji – w przeciwieństwie do klasycznej odległości euklidesowej, która nie radzi sobie ze zmiennymi nienumerycznymi.

- **Zmienne numeryczne**: dla każdej pary punktów obliczana jest różnica absolutna, znormalizowana do zakresu [0, 1], czyli względem rozpiętości danej cechy.
- **Zmienne kategoryczne**: porównywane są wartości – jeśli są identyczne, podobieństwo wynosi 1, w przeciwnym razie 0.
- **Średnia ważona**: na końcu Gower wylicza średnią tych podobieństw dla wszystkich cech, uzyskując wartość końcową odwzorowującą dystans (im mniejszy, tym dane są bardziej podobne).

Metryka ta pozwala na bezproblemowe uwzględnienie wszystkich typów danych bez potrzeby ich sztucznego przekształcania. Z tego względu została zastosowana jako podstawa do tworzenia macierzy odległości, wykorzystywanej następnie w algorytmach klasteryzacji (np. aglomeracyjnej) i ocenie jakości klastrów.


In [ ]:
distances = gower_matrix(data,cat_features=cat_features_array)

#### Metoda łokcia

Jest to metoda służąca do znalezienia optymalnej liczby klastrów (k) w algorytmie k-średnich. Na osi X znajduje się liczba klastrów (k), a na osi Y wartość metryki WCSS (Within-Cluster Sum of Squares – suma kwadratów odległości punktów od centroidów klastrów). Optymalna wartość k to punkt, w którym wykres zaczyna się wypłaszczać – przypomina kształt łokcia, co oznacza, że dalsze zwiększanie liczby klastrów nie przynosi istotnego zmniejszenia WCSS.

In [ ]:
wcss = []
silhouette = []
for j in range(1, 15):
    kp = KPrototypes(n_clusters=j, random_state=42, init='random', n_init=10)
    kp.fit(data, categorical=categorical_indices)
    wcss.append(kp.cost_)
    kp_labels = kp.labels_
    if j != 1:
        silhouette.append(silhouette_score(distances, kp_labels, metric='precomputed'))

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(range(1, 15), wcss, marker='o', markerfacecolor='red', markeredgecolor='black', markersize=5)
plt.xlabel('Liczba klastrów k', labelpad=10, fontsize=12)
plt.ylabel('Wartość WCSS', labelpad=10, fontsize=12)
plt.title('Metoda łokcia', pad=20, fontsize=14)


plt.subplot(1, 2, 2)
plt.plot(range(2, 15), silhouette[:], marker='o', markerfacecolor='red', markeredgecolor='black', markersize=5)
plt.xlabel('Liczba klastrów k', labelpad=10, fontsize=12)
plt.ylabel('Wartość Silhouette', labelpad=10, fontsize=12)
plt.title('Współczynnik Silhouette', pad=20, fontsize=14)

plt.show()

#### Dendrogram

Jest to forma wizualizacji procesu łączenia klastrów w metodzie aglomeracyjnej. Liście reprezentują pojedyncze punkty danych. Gałęzie pokazują, w jaki sposób i na jakim poziomie (odległości) punkty lub grupy punktów zostały połączone. Wysokość połączenia oznacza odległość między łączonymi grupami.

- oś X -> etykiety punktów lub identyfikatory klastrów.
- oś Y -> wartość dystansu przy łączeniu klastrów. Im wyższy poziom połączenia, tym mniej podobne były klastry.

In [ ]:
agglomerative_1 = AgglomerativeClustering(n_clusters=None, distance_threshold=0, linkage='average',metric='precomputed').fit(distances)

def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    dendrogram(linkage_matrix, **kwargs)
plt.figure(figsize=(14, 10))

plt.title("Dendrogram dla metody aglomeracyjnej")
plot_dendrogram(agglomerative_1, truncate_mode="level", p=3)
plt.xlabel("Indeksy liści dendrogramu (próbki lub połączone klastry)")
plt.grid(False)
plt.show()

#### Wykres zmiany wartości współczynnika zarysu dla metody aglomeracyjnej dla różnej liczby klastrów

In [ ]:
silhouette = []
for j in range(1, 15):
    ag = AgglomerativeClustering(n_clusters=j, distance_threshold=None, linkage='average',metric='precomputed')
    ag.fit(distances)

    ag_labels = ag.labels_
    if j != 1:
        silhouette.append(silhouette_score(distances, ag_labels, metric='precomputed'))

plt.figure(figsize=(14, 6))


plt.plot(range(2, 15), silhouette[:], marker='o', markerfacecolor='red', markeredgecolor='black', markersize=5)
plt.xlabel('Liczba klastrów k', labelpad=10, fontsize=12)
plt.ylabel('Wartość Silhouette', labelpad=10, fontsize=12)
plt.title('Współczynnik Silhouette', pad=20, fontsize=14)

plt.show()

#### Wybór liczby klastrów

In [ ]:
agglomerative = AgglomerativeClustering(n_clusters=3, linkage='average', metric='precomputed')
kprototypes = KPrototypes(n_clusters=3, random_state=42, init='random', n_init=10)
agglomerative.fit(distances)
kprototypes.fit(data, categorical=categorical_indices)

agglomerative_labels = agglomerative.labels_
kprototypes_labels = kprototypes.labels_

#### Współczynnik zarysu (ang. Silhouette coefficient)

<b>Użyta implementacja z sklearn oblicza średnią wartości współczynnika zarysu.</b>

Jest to metryka wskazująca jak dobrze punkt pasuje do klastra w którym się znajduje.

Interpretacja wyników:
- silhouette = 1 -> najlepsza wartość
- silhouette = 0 -> nakładające się klastry
- silhouette = -1 -> najgorsza wartość

Jeśli wartości są ujemne, oznacza to błąd w przyporządkowaniu obserwacji do klastra. Inny klaster jest lepszy.

In [ ]:
silhouette = (silhouette_score(distances, agglomerative_labels, metric='precomputed'), silhouette_score(distances, kprototypes_labels, metric='precomputed'))
silhouette

#### Macierz kontyngencji dla KPrototypes, AgglomerativeClustering

Pokazuje jak często występują różne kombinacje wartości dwóch zmiennych kategorycznych. Pomaga ocenić, czy algorytm poprawnie przypisał rzeczywiste próbki do otrzymanych grup.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12, 10))

contingency_mat = [contingency_matrix(status, agglomerative_labels), contingency_matrix(status, kprototypes_labels)]
titles = ['dla metody aglomeracyjnej', 'dla metody KPrototypes']

for i, mat in enumerate(contingency_mat):
    ax[i].imshow(
        X=mat,
        cmap='Accent',
    )
    ax[i].set_xlabel('Przewidziane etykiety', fontsize=12, labelpad=10)
    ax[i].set_ylabel('Rzeczywiste etykiety', fontsize=12, labelpad=10)
    ax[i].set_xticks(ticks=np.arange(mat.shape[0]), labels=[0, 1, 2])
    ax[i].set_yticks(ticks=np.arange(mat.shape[0]), labels=status_encoder.classes_)
    ax[i].set_title(f'Macierz kontyngencji {titles[i]}', fontsize=14, pad=20)
    ax[i].grid(False)

    for j in range(mat.shape[0]):
        for k in range(mat.shape[1]):
            ax[i].text(k, j, mat[j, k], ha='center', va='center', color='white', fontsize=12)

#### Indeks Randa

Indeks Randa jest miarą służącą do ewaluacji jakości grupowania (klastrowania) poprzez porównanie dwóch różnych podziałów tego samego zbioru danych. Jest to popularna metoda oceny zgodności między algorytmicznym grupowaniem a znaną klasyfikacją referencyjną.

Główne cechy Indeksu Randa:

- Mierzy podobieństwo między dwoma grupowaniami na podstawie par obiektów
- Wartość zawiera się w przedziale [0,1], gdzie 1 oznacza całkowitą zgodność grupowań
- Uwzględnia zarówno pary obiektów przypisane do tej samej grupy, jak i pary przypisane do różnych grup

In [ ]:
rand = (rand_score(status, agglomerative_labels), rand_score(status, kprototypes_labels))

rand

#### Współczynnik wzajemnej informacji (ang. Mutual Information, MI)

Zakres wartości: mi >= 0

mi = 0 -> brak relacji miedzy obiema zmiennymi

Im większy wynik tym silniejsza relacja.


#### Znormalizowany MI

Zakres wartości: 0 >= nmi <= 1

mi = 0 -> brak relacji między obiema zmiennymi

mi = 1 -> bardzo silna relacja między obiema zmiennymi

In [ ]:
hierarchical_mi = mutual_info_score(status, agglomerative_labels)
hierarchical_nmi = normalized_mutual_info_score(status, agglomerative_labels)

(hierarchical_mi, hierarchical_nmi)

In [ ]:
kmeans_mi = mutual_info_score(status, kprototypes_labels)
kmeans_nmi = normalized_mutual_info_score(status, kprototypes_labels)

(kmeans_mi, kmeans_nmi)

#### Podział wartości zmiennych według grup zwróconych przez KPrototypes

In [ ]:
data['Status'] = kprototypes_labels
final_columns = number_columns.values
final_columns = np.append(final_columns, 'Status')

plt.figure(figsize=(15, 10))
sns.pairplot(data[final_columns], hue='Status', palette='viridis')
plt.tight_layout()
plt.show()

#### Podział wartości zmiennych według grup zwróconych przez AgglomerativeClustering

In [ ]:
data['Status'] = agglomerative_labels

plt.figure(figsize=(15, 10))
sns.pairplot(data[final_columns], hue='Status', palette='viridis')
plt.tight_layout()
plt.show()

#### Podział wartości zmiennych według rzeczywistych etykiet klas

In [ ]:
data['Status'] = status

plt.figure(figsize=(15, 10))
sns.pairplot(data[final_columns], hue='Status', palette='viridis')
plt.tight_layout()
plt.show()